In [4]:
import pandas as pd
from pathlib import Path
import os
import sys
import numpy as np

In [5]:
root = Path('/hpc/group/kamaleswaranlab/EmoryDataset/Images/chest_xrays')

YEARS = [2015, 2016, 2017, 2018, 2019, 2020, 2021]
images= "extracted-images"
metadata = "metadata_with_supertables.dsv"


#### Combine meta data of all years

In [72]:
for i in range(len(YEARS)):
    metadf_ = pd.read_csv(root / str(YEARS[i]) / metadata, sep = '|')
    metadf_['year'] = [YEARS[i]]*len(metadf_) 
    if i > 0:
        metadf = pd.concat((metadf, metadf_), axis = 0)
    else:
        metadf = metadf_.copy()

print(metadf.shape)

(261428, 28)


In [73]:
metadf.year.unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2021])

#### Filter out accession numbers that are not matched to supertables. 

In [74]:
metadf.supertable_path.isna().sum()/len(metadf) # Just 0.2% of the images, so not worth spending time debugging

np.float64(0.0029338861942867635)

In [75]:
metadfsups = metadf.loc[metadf.supertable_path.notna()]

In [76]:
metadfsups.to_csv(root / 'metadata_with_supertables_filtered.csv', index = False)

### Notes

In [17]:
metadf = pd.read_csv(root / 'metadata_with_supertables_filtered.csv')

/tmp/ipykernel_1827307/3460263475.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  metadf = pd.read_csv(root / 'metadata_with_supertables_filtered.csv')


In [26]:
notes_files = list(root.glob('IBJG_CXR_ACC_*'))

notes1 = pd.read_csv(notes_files[0], sep = '|')
notes2 = pd.read_csv(notes_files[1], sep = '|')
notes = pd.concat((notes1, notes2), axis = 0)

In [29]:
notes.to_csv(root / 'all_notes.csv')

In [30]:
notes.columns

Index(['ACC_NBR', 'PATIENT_ID', 'EMPI_NBR', 'ENCNTR_ID',
       'HNAM_DOCUMENT_CLINICAL_ID', 'HNAM_DOCUMENT_CLINICAL_NM',
       'DAY_VERIFIED', 'DOC_TEXT', 'DOC_ABSTRACT', 'EVENT_DOCUMENT_DESC',
       'EVENT_DOCUMENT_KEY'],
      dtype='object')

In [43]:
metadf = metadf.loc[metadf.AccessionNumber.isin(notes.ACC_NBR)]

In [44]:
metadf.to_csv( root / 'metadata_with_supertables_filtered_notes_filtered.csv', index = False)